In [ ]:
!pip install tensorflow --upgrade
!pip install keras --upgrade
!pip install --upgrade tensorflow-metadata


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall, AUC

# Loading the training dataset
url = 'https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/hau/train.tsv'
df = pd.read_csv(url, sep='\t')

# Extract the reviews and labels from the training dataset
reviews = df['tweet'].values
labels = df['label'].values

# Spliting the training dataset into training and validation sets
train_reviews, val_reviews, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Tokenizing the training reviews
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_reviews)
max_len = max(len(x) for x in sequences)
train_data = pad_sequences(sequences, maxlen=max_len)

# Tokenizing the validation reviews
val_sequences = tokenizer.texts_to_sequences(val_reviews)
val_data = pad_sequences(val_sequences, maxlen=max_len)

# One-hot encoding the labels for training and validation sets
train_labels = np.array(pd.get_dummies(train_labels))
val_labels = np.array(pd.get_dummies(val_labels))


# Building the model
model = Sequential()
model.add(Embedding(5000, 32, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Adding precision, recall, and F1 score metrics to the model
precision = Precision(name='precision')
recall = Recall(name='recall')
auc = AUC(name='auc')

# Creating a function to compute the F1 score
def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r+1e-7))

# Compiling the model with the new metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall, f1_score, auc])

# Training the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=32)


Epoch 1/5
355/355 [==============================] - 6s 14ms/step - loss: 0.8820 - accuracy: 0.5676 - precision: 0.6851 - recall: 0.3407 - f1_score: 0.2492 - auc: 0.7670 - val_loss: 0.7665 - val_accuracy: 0.6434 - val_precision: 0.7350 - val_recall: 0.4744 - val_f1_score: 0.5757 - val_auc: 0.8339
Epoch 2/5
355/355 [==============================] - 4s 10ms/step - loss: 0.6157 - accuracy: 0.7424 - precision: 0.7752 - recall: 0.6855 - f1_score: 0.7061 - auc: 0.8971 - val_loss: 0.6645 - val_accuracy: 0.7101 - val_precision: 0.7330 - val_recall: 0.6603 - val_f1_score: 0.6880 - val_auc: 0.8802
Epoch 3/5
355/355 [==============================] - 3s 10ms/step - loss: 0.4289 - accuracy: 0.8373 - precision: 0.8545 - recall: 0.8171 - f1_score: 0.8459 - auc: 0.9504 - val_loss: 0.7207 - val_accuracy: 0.6966 - val_precision: 0.7136 - val_recall: 0.6670 - val_f1_score: 0.6904 - val_auc: 0.8714
Epoch 4/5
355/355 [==============================] - 4s 13ms/step - loss: 0.2975 - accuracy: 0.8964 - prec